# Visualisation des données

In [2]:
import visdom
import numpy as np
import sys
sys.path.append("../TimeSeriesTools")
import utils
from cerberus import Validator
import chart_studio.plotly as py
import plotly.express as px
import plotly.tools as tls
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

## Connexion à la base MongoDB

In [3]:
import mongodb_utils
db_host= 'localhost'
port = '27018'
db_name='TimeSeriesBench'
mongodb_client = mongodb_utils.mongodb_connect(db_host, port)


Trying to connect to MongoDB server: localhost on port: 27018


In [4]:
import kairosdb_utils
global kairosdb_server 
kairosdb_server = "http://localhost:9080"

In [5]:
import influxdb_utils
db_host= 'localhost'
port = '8086'
db_name='TimeSeriesBench'
influxdb_client = influxdb_utils.influxdb_connect(db_host, port)

Trying to connect to InfluxDB server without proxy: localhost on port: 8086
connection sucess!


## Lecture des données dans la base
Cette fonction est utilisée pour lire tous les documents dans la base de mongodb, mais il reste un problème : mongodb n'est pas une base de données spécialement conçue pour les données de séries temporaires, donc c'est possible d'exister certains données dédoublées avec même timestamps. Soit on définit un règle avant le stockage de mongodb pour supprimer ou modifier les données dédboulées, ou soit on le néglige.

In [6]:
def get_collection_scheme(db_name,scheme_name):
    db = mongodb_client[db_name]
    schemes_coll = db['schemes']
    scheme = schemes_coll.find({"name":scheme_name})
    return scheme

In [7]:
def mongodb_find_all_data(db_name,coll_name,scheme):
    data = mongodb_utils.get_all_data(mongodb_client,db_name,coll_name,scheme)
    return data

In [8]:
def mongodb_find_data_select_by_tags(db_name,coll_name,tags,scheme):
    data = mongodb_utils.get_data_select_by_tags(mongodb_client,db_name,coll_name,tags,scheme)
    return data

In [9]:
def kairosdb_find_all_data(db_name,coll_name,scheme):
    data = kairosdb_utils.get_all_data(kairosdb_server,db_name,coll_name,scheme)
    return data

In [10]:
def kairosdb_find_data_select_by_tags(db_name,coll_name,tags,scheme):
    data = kairosdb_utils.get_data_select_by_tags(kairosdb_server,db_name,coll_name,tags,scheme)
    return data

In [11]:
def influxdb_find_all_data(db_name,coll_name,scheme):
    data = influxdb_utils.get_all_data(influxdb_client,db_name,coll_name,scheme)
    return data

In [12]:
def influxdb_find_data_select_by_tags(db_name,coll_name,tags,scheme):
    data = influxdb_utils.get_data_select_by_tags(influxdb_client,db_name,coll_name,tags,scheme)
    return data

## Conversion les dates des données à unix timestamp
Les objects de datetime ne peuvent pas directemant utilisés pour tracer l'axis X, donc il faut d'abord convertir les datetime à unix timestamp.

In [13]:
def to_unix_time(dt):
    epoch =  datetime.utcfromtimestamp(7200)
    return (dt - epoch).total_seconds() * 1000

In [14]:
def str_to_unix(date):
    dt = datetime.strptime(date, '%d/%m/%Y %H:%M:%S')
    epoch = datetime.utcfromtimestamp(0)
    return int((dt - epoch).total_seconds()) * 1000

## Nettoyage des données dans la base
Normalement cette étape est faite avant le stockage des données, vu que on ne sait pas encore les règles détaillés pour la validation les données, on insère quand même les données corruptibles dans la base. <br>
Dans cet exemple, la source des données contient une ligne de données qui manque trois champs pour tester la validation des données.<br>
Donc avant visualiser les courbes, on trouve d'abord les données corruptibles et les supprime. 
Les règles de validation temporaires sont : 
* tous les champs sont présentés
* les types des champs sont string sauf "_id" car cerberus ne connaît pas le type `bson.objectid.ObjectId`
* tous les champs ne sont pas string vide

In [15]:
def clean_data(scheme,data):
    from cerberus import Validator
    v = Validator(scheme)
    for index,item in enumerate(data,start=0):
        res = v.validate(item)
        if (res == False):
            print("corrupt data in line :",index,", error : ",v.errors)
            del data[index]

### Connecter à la base de test et affichhe le nombre total des données trouvées dans la collection désirée

In [16]:
scheme = get_collection_scheme(db_name,'SmartGrid')
scheme[0]

{'_id': ObjectId('5ed8af9859e948764cadceb2'),
 'name': 'SmartGrid',
 'value': {'_id': {'required': True},
  'timestamp': {'type': 'string', 'required': True, 'empty': False},
  'tagname': {'type': 'string', 'required': True, 'empty': False},
  'value': {'type': 'string', 'required': True, 'empty': False},
  'quality': {'type': 'string', 'required': True, 'empty': False}}}

In [ ]:
%%time
coll_name='SmartGridCryolite20190101OneMonthV10000'
data = mongodb_find_all_data(db_name,coll_name,scheme[0]['value'])
print("number of docs",len(data))

In [18]:
%%time
coll_name='SmartGridCryolite20190101OneMonthV10000'
tags = { 'Buiding' : 'CRY', 'Device' : 'CENTRALE_SOLAIRE', 'Measure' : 'CRY_act_prod_pow' }
data = mongodb_find_data_select_by_tags(db_name,coll_name,tags,scheme[0]['value'])
print("number of docs",len(data))

27584  documents found
number of docs 27584
CPU times: user 128 ms, sys: 17.9 ms, total: 146 ms
Wall time: 463 ms


In [21]:
%%time
coll_name='SmartGridCryolite20190101OneMonthV10000'
tags = { 'Buiding' : 'CRY', 'Device' : 'CENTRALE_SOLAIRE', 'Measure' : 'CRY_act_prod_pow' }
data = kairosdb_find_data_select_by_tags(db_name,coll_name,tags,scheme[0]['value'])
#data = kairosdb_find_all_data(db_name,coll_name,scheme[0]['value'])
print("number of docs",len(data))

number of docs 27584
CPU times: user 152 ms, sys: 23.3 ms, total: 176 ms
Wall time: 472 ms


In [20]:
%%time
coll_name='SmartGridCryolite20190101OneMonthV10000'
tags = { 'Buiding' : 'CRY', 'Device' : 'CENTRALE_SOLAIRE', 'Measure' : 'CRY_act_prod_pow' }
data = influxdb_find_data_select_by_tags(db_name,coll_name,tags,scheme[0]['value'])
print("number of docs",len(data))

number of docs 27584
CPU times: user 2.85 s, sys: 266 ms, total: 3.11 s
Wall time: 7.15 s


### Nettoyer les données récupérées
Les informations des données corruptibles seront affichées dans la console :
* position (nbr de ligne)
* nom de champs qui a des problème
* message d'erreur, pourquoi cette partie de données n'est pas validée

In [ ]:
clean_data(scheme[0]['value'],data)

In [22]:
%%time
df = pd.DataFrame(data)
df[0:5]

CPU times: user 33.3 ms, sys: 1.23 ms, total: 34.5 ms
Wall time: 33.4 ms


,_id,timestamp,tagname,value,quality
0,0,1546334112000,TAG.CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow.all....,1.000000000,100.0
1,1,1546334118000,TAG.CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow.all....,0.000000000,100.0
2,2,1546334137000,TAG.CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow.all....,1.000000000,100.0
3,3,1546334143000,TAG.CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow.all....,0.000000000,100.0
4,4,1546334153000,TAG.CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow.all....,1.000000000,100.0


In [ ]:
df['value']

In [ ]:
df.tagname.unique()

In [ ]:
df2 = df.loc[df['tagname'] == 'CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow']
df2[0:10]

### Convertir les string d'heure à datetime object 

In [ ]:
# convertir cas MongoDB
df['timestamp'] = df['timestamp'].apply(str_to_unix)

### Préparer la figure basée sur la bibliothèque `plotly`
Pour éviter les données dédoublées, on recupère les premières 5000 lignes comme la source de données du plot.<br>
Dans cet plot les courbes partagent les axis-X et axis-Y, donc on peut observer que les tendances des courbes ne sont pas très claire parce que les plages de valeur des colonnes de données sont très variées.<br>

In [ ]:
# Cas MongoDB
x = df2['timestamp'].values
nb_pts = len(x)
nb_pts

In [ ]:
y_value=[float(item) for item in df2["value"]]
y_quality=[float(item)/100 for item in df2["quality"]]

In [ ]:
x = df['timestamp'].values
nb_pts = len(x)
nb_pts

In [ ]:
y_value=[float(item) for item in df["value"]]
y_quality=[float(item)/100 for item in df["quality"]]

In [ ]:
%matplotlib inline
#nb_pts=700000
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=x,
                y=y_value,
                name="value",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=x,
                y=y_quality,
                name="quality",
                line_color='dimgray',
                opacity=0.8))

# Use date string to set xaxis range
fig.update_layout(xaxis_range=[x[0],
                               x[nb_pts-1]],
                  title_text="smartgrid data series")
fig.show()

Cet plot contient trois subplots, dans la figure, les courbes partagent l'axis-X mais elles possèdent différents axis-Y.<br>
On remarque que les tendances des courbes sont plus évidentes que la figure précédente.

In [ ]:
fig = make_subplots(
    rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.02
)

fig.add_trace(go.Scatter(
                x=x,
                y=y_value,
                name="value",
                line_color='deepskyblue',
                line_width = 2,
                opacity=0.8),
              row=1, col=1)

fig.update_layout(height=1000, width=1000,
                  title_text="smartgrid data series")
fig.show()

### Initialiser visdom et envoyer le plot à visdom via requête Http post
Avant cette étape, il faut lancer le service de visdom dans le terminal.
```bash
> visdom
```
L'address du web UI de visdom sera affichée dans la console.

In [ ]:
vis = visdom.Visdom()
vis.plotlyplot(fig, win="mywin3")

### Ajuster la taille de la fenêtre

In [ ]:
vis.update_window_opts(win = "mywin3", opts=dict(width=1200, height=1500))

Après l'exécution de ce script, ouvrir le web UI de visdom, le plot est présenté dans une fenêtre intéractive.